<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza y Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano y Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Michelle Avendaño
- Nombre de alumno 2: Claudia Navarro


### **Link de repositorio de GitHub:** `https://github.com/ClaudiaRayen/MDS7202-Laboratorios/tree/Lab8`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [ ]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones.

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.csv`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = 'Dirección donde tiene los archivos en el Drive'
except:
    print('Ignorando conexión drive-colab')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_players = pd.read_csv('/content/drive/MyDrive/lab8/stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego.

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país.

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [ ]:

df_players['Seleccionado'] = ~df_players.loc[:,'National_Position'].isna()*1
df_players['Seleccionado'].value_counts()


0    16513
1     1075
Name: Seleccionado, dtype: int64

Se crea una nueva columna llamada 'Seleccionado' que contiene 1 si la columna 'National_Position' no es un valor nulo y 0 si lo es. Luego, se
cuentan los valores que fueron seleccionados y no seleccionados. En donde los seleccionados son 1075 y los no seleccionados 16513.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores.

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.

En primer lugar, se analizan los diferentes tipos de variables y se clasifican en cuatro categorías: texto, numéricas, categóricas u ordinales. A continuación, se decide aplicar el Standard Scaler a las variables numéricas, dado que su distribución se asemeja a una distribución normal. Por otro lado, para las columnas categóricas se opta por utilizar el One-Hot Encoder, ya que representan categorías sin un orden específico. Por último, se elige el Ordinal Encoder para la variable 'Week_foot', ya que esta variable implica un orden específico relacionado con el mejor o peor uso de la pierna débil.

In [ ]:
numericas = ['Age','Weight','Height','Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys']
categoricas = ['Preffered_Foot','Work_Rate','Skill_Moves']
ordinales = ['Weak_foot']
texto = ['Name','Club_Position','Nationality','National_Position','Seleccionado']

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ct = ColumnTransformer(
    [
        ('Scaler',MinMaxScaler(),numericas),
        ('One Hot',OneHotEncoder(sparse=False, handle_unknown="ignore",drop='if_binary'),categoricas),
        ('Ordinal',OrdinalEncoder(),ordinales)
    ],
    remainder='passthrough'
)

Siguiendo el gráfico proporcionado para la selección de modelos, se observa que hay menos de 100,000 ejemplos disponibles. En este contexto, se opta por utilizar un modelo de Linear SVC debido a la naturaleza de los datos. Además, como los datos no son de tipo texto, se selecciona K Nearest Neighbor. Por último, se elige implementar un ensamblado de árboles, específicamente un Random Forest, como estrategia de modelado.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

pip_svc = Pipeline(steps=[
                      ("preprocessing", ct),
                      ("model",SVC(kernel='linear',random_state=42) )
])

pip_knn = Pipeline(steps=[
                      ("preprocessing", ct),
                      ("model",KNeighborsClassifier() )
])

pip_rf = Pipeline(steps=[
                      ("preprocessing", ct),
                      ("model",RandomForestClassifier(random_state=42) )
])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [ ]:
labels = df_players.loc[:, "Seleccionado"]
features = df_players.drop(columns=texto)
#80% entrenamiento y 20% test
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.2,random_state=42,stratify=labels)

SVC

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import prettytable as pt
import warnings

warnings.filterwarnings("ignore")

# Fit
pip_svc.fit(X_train, y_train)

# Predict
y_pred = pip_svc.predict(X_train)

# Matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Informe de clasificación
classification_rep = classification_report(y_train, y_pred, zero_division=1)

# F1-Score
f1 = f1_score(y_train, y_pred).round(2)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table.add_row(["Real Negativa", conf_matrix[0, 0], conf_matrix[0, 1]])
table.add_row(["Real Positiva", conf_matrix[1, 0], conf_matrix[1, 1]])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con svc:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con svc:")
print(classification_rep)

print(f"\nF1-Score en conjunto de entrenamiento con svc: {f1}")


Matriz de confusión en conjunto de entrenamiento con svc:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |        13210        |          0          |
| Real Positiva |         860         |          0          |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de entrenamiento con svc:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     13210
           1       1.00      0.00      0.00       860

    accuracy                           0.94     14070
   macro avg       0.97      0.50      0.48     14070
weighted avg       0.94      0.94      0.91     14070


F1-Score en conjunto de entrenamiento con svc: 0.0


In [ ]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

# Predict en conjunto de prueba
y_pred_test = pip_svc.predict(X_test)

# Matriz de confusión
conf_matrix_test = confusion_matrix(y_test, y_pred_test)

# Informe de clasificación
classification_rep_test = classification_report(y_test, y_pred_test, zero_division=1)

# F1-Score
f1_test = f1_score(y_test, y_pred_test).round(2)

# Crear una tabla para la matriz de confusión
table_test = pt.PrettyTable()
table_test.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table_test.add_row(["Real Negativa", conf_matrix_test[0, 0], conf_matrix_test[0, 1]])
table_test.add_row(["Real Positiva", conf_matrix_test[1, 0], conf_matrix_test[1, 1]])

# Imprimir resultados en conjunto de prueba
print("Matriz de confusión en conjunto de prueba con svc:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con svc:")
print(classification_rep_test)

print(f"\nF1-Score en conjunto de prueba con svc: {f1_test}")


Matriz de confusión en conjunto de prueba con svc:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |         3303        |          0          |
| Real Positiva |         215         |          0          |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de prueba con svc:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3303
           1       1.00      0.00      0.00       215

    accuracy                           0.94      3518
   macro avg       0.97      0.50      0.48      3518
weighted avg       0.94      0.94      0.91      3518


F1-Score en conjunto de prueba con svc: 0.0


KNN

In [ ]:
warnings.filterwarnings("ignore")

# Fit
pip_knn.fit(X_train, y_train)

# Predict
y_pred = pip_knn.predict(X_train)

# Matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Informe de clasificación
classification_rep = classification_report(y_train, y_pred, zero_division=1)

# F1-Score
f1 = f1_score(y_train, y_pred).round(2)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table.add_row(["Real Negativa", conf_matrix[0, 0], conf_matrix[0, 1]])
table.add_row(["Real Positiva", conf_matrix[1, 0], conf_matrix[1, 1]])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con knn:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con knn:")
print(classification_rep)

print(f"\nF1-Score en conjunto de entrenamiento con knn: {f1}")


Matriz de confusión en conjunto de entrenamiento con knn:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |        13170        |          40         |
| Real Positiva |         740         |         120         |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de entrenamiento con knn:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     13210
           1       0.75      0.14      0.24       860

    accuracy                           0.94     14070
   macro avg       0.85      0.57      0.60     14070
weighted avg       0.93      0.94      0.93     14070


F1-Score en conjunto de entrenamiento con knn: 0.24


In [ ]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

# Predict en conjunto de prueba
y_pred_test = pip_knn.predict(X_test)

# Matriz de confusión
conf_matrix_test = confusion_matrix(y_test, y_pred_test)

# Informe de clasificación
classification_rep_test = classification_report(y_test, y_pred_test, zero_division=1)

# F1-Score
f1_test = f1_score(y_test, y_pred_test).round(2)

# Crear una tabla para la matriz de confusión
table_test = pt.PrettyTable()
table_test.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table_test.add_row(["Real Negativa", conf_matrix_test[0, 0], conf_matrix_test[0, 1]])
table_test.add_row(["Real Positiva", conf_matrix_test[1, 0], conf_matrix_test[1, 1]])

# Imprimir resultados en conjunto de prueba
print("Matriz de confusión en conjunto de prueba con knn:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con knn:")
print(classification_rep_test)

print(f"\nF1-Score en conjunto de prueba con knn: {f1_test}")


Matriz de confusión en conjunto de prueba con knn:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |         3283        |          20         |
| Real Positiva |         202         |          13         |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de prueba con knn:
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      3303
           1       0.39      0.06      0.10       215

    accuracy                           0.94      3518
   macro avg       0.67      0.53      0.54      3518
weighted avg       0.91      0.94      0.91      3518


F1-Score en conjunto de prueba con knn: 0.1


rf

In [ ]:
warnings.filterwarnings("ignore")

# Fit
pip_rf.fit(X_train, y_train)

# Predict
y_pred = pip_rf.predict(X_train)

# Matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Informe de clasificación
classification_rep = classification_report(y_train, y_pred, zero_division=1)

# F1-Score
f1 = f1_score(y_train, y_pred).round(2)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table.add_row(["Real Negativa", conf_matrix[0, 0], conf_matrix[0, 1]])
table.add_row(["Real Positiva", conf_matrix[1, 0], conf_matrix[1, 1]])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con rf:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con rf:")
print(classification_rep)

print(f"\nF1-Score en conjunto de entrenamiento con rf: {f1}")


Matriz de confusión en conjunto de entrenamiento con rf:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |        13210        |          0          |
| Real Positiva |          1          |         859         |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de entrenamiento con rf:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13210
           1       1.00      1.00      1.00       860

    accuracy                           1.00     14070
   macro avg       1.00      1.00      1.00     14070
weighted avg       1.00      1.00      1.00     14070


F1-Score en conjunto de entrenamiento con rf: 1.0


In [ ]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

# Predict en conjunto de prueba
y_pred_test = pip_rf.predict(X_test)

# Matriz de confusión
conf_matrix_test = confusion_matrix(y_test, y_pred_test)

# Informe de clasificación
classification_rep_test = classification_report(y_test, y_pred_test, zero_division=1)

# F1-Score
f1_test = f1_score(y_test, y_pred_test).round(2)

# Crear una tabla para la matriz de confusión
table_test = pt.PrettyTable()
table_test.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table_test.add_row(["Real Negativa", conf_matrix_test[0, 0], conf_matrix_test[0, 1]])
table_test.add_row(["Real Positiva", conf_matrix_test[1, 0], conf_matrix_test[1, 1]])

# Imprimir resultados en conjunto de prueba
print("Matriz de confusión en conjunto de prueba con rf:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con rf:")
print(classification_rep_test)

print(f"\nF1-Score en conjunto de prueba con rf: {f1_test}")


Matriz de confusión en conjunto de prueba con rf:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |         3295        |          8          |
| Real Positiva |         194         |          21         |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de prueba con rf:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      3303
           1       0.72      0.10      0.17       215

    accuracy                           0.94      3518
   macro avg       0.83      0.55      0.57      3518
weighted avg       0.93      0.94      0.92      3518


F1-Score en conjunto de prueba con rf: 0.17


**Respuesta**:
AL entrenar y evaluar los 3 modelos, se ve el resultado de las distintas metricas. En donde:

Los valores de accuracy miden la proporción de ejemplos clasificados correctamente (tanto la clase 0 como la clase 1) con respecto al total de ejemplos.

$$
\text{Accuracy} = \frac{TP + TN}{TP + FN + TN + FP}
$$

La precisión mide la proporción de ejemplos clasificados como positivos que son verdaderos positivos en comparación con todos los ejemplos clasificados como positivos.
$$
\text{Precision} = \frac{TP}{TP + FP}
$$

El recall mide la proporción de ejemplos positivos que fueron clasificados correctamente como positivos en comparación con todos los ejemplos reales positivos.
$$
\text{Recall} = \frac{TP}{TP + FN}
$$

El F1-Score es una métrica que combina tanto la precisión como el recall. El F1-Score es útil cuando se desea tener una única métrica que combine tanto la capacidad de un modelo para clasificar correctamente ejemplos positivos (precision) como su capacidad para capturar todos los ejemplos positivos (recall).

$$
\text{F1-Score} = 2 \times \frac{\text{Precision} \times \text{Recall}}{\text{Precision} + \text{Recall}}
$$

Donde:
- \(TP\) es el número de verdaderos positivos.
- \(TN\) es el número de verdaderos negativos.
- \(FP\) es el número de falsos positivos.
- \(FN\) es el número de falsos negativos.

Luego, para comparar los tres modelos anteriores en términos de su capacidad para clasificar la clase 1 (que sea selecionado), podemos observar el valor del puntaje F1 para la clase 1 en cada modelo. Cuanto mayor sea el puntaje F1, mejor será el rendimiento del modelo para la clase 1. En donde los valores:

1. Modelo SVC:
   - Puntaje F1 para la clase 1: 0.00

2. Modelo K-Nearest Neighbors (KNN):
   - Puntaje F1 para la clase 1: 0.10

3. Modelo Random Forest (RF):
   - Puntaje F1 para la clase 1: 0.17

Podemos decir que el Modelo Random Forest (RF) tiene el mejor rendimiento en la clasificación de la clase 1, seguido por el Modelo K-Nearest Neighbors (KNN). El Modelo de Máquinas de Soporte Vectorial (SVC) tiene un puntaje F1 muy bajo para la clase 1, lo que indica que no es efectivo en la clasificación

Aun asi, los tres modelos tienen desafíos en la clasificación de la clase 1. Aunque el modelo SVC tiene una precisión perfecta para la clase 1, su recall es muy bajo, lo que indica que no puede identificar la mayoría de los casos verdaderos positivos. El modelo Random Forest tiene un rendimiento relativamente mejor en términos de precisión y recall en comparación con los otros dos modelos. Sin embargo, en general, el rendimiento de todos los modelos es limitado en la clasificación de la clase 1.  

Finalmente, los resultados actuales pueden mejorarse, teniendo en cuenta que los modelos fueron entrenados con parámetros por defecto, sin utilizar validación cruzada ni una métrica de evaluación específica para optimizar el rendimiento. Además, se utilizó un conjunto de datos de entrenamiento desbalanceado, lo que influyó en que el modelo se centrara en clasificar con mayor precisión los ejemplos de la clase 0, es decir, aquellos que no fueron seleccionados.

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas.

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características.

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF']
central_ataque = ['RW', 'CAM', 'LW']
central = ['RM', 'CM', 'LM']
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [ ]:
#clases
position_encoder = {  'ST':'ataque',
           'CF':'ataque',

           'LF':'ataque',
           'RF':'ataque',
           'LS':'ataque',
           'RS':'ataque',

           'RW':'central_ataque',
           'CAM':'central_ataque',
           'LW':'central_ataque',

           'RAM':'central_ataque',
           'LAM':'central_ataque',

           'RM':'central',
           'CM':'central',
           'LM':'central',

           'LCM':'central',
           'RCM':'central',

           'RWB':'central_defensa',
           'CDM':'central_defensa',
           'LWB':'central_defensa',

           'LDM':'central_defensa',
           'RDM':'central_defensa',

           'RB':'defensa',
           'CB':'defensa',
           'LB':'defensa',

           'RCB':'defensa',
           'LCB':'defensa',

           'GK':'arquero'
}

In [ ]:
df_players = df_players.replace({'Club_Position': position_encoder})
df_filtered = df_players[(df_players["Club_Position"] != "Sub") & (df_players["Club_Position"] != "Res")]
df_filtered['Club_Position'].value_counts()


defensa            2444
central            1614
ataque              865
central_defensa     741
central_ataque      653
arquero             632
Name: Club_Position, dtype: int64

Siguiendo el gráfico proporcionado para la selección de modelos, se observa que hay menos de 100,000 ejemplos disponibles. En este contexto, se opta por utilizar un modelo de Linear SVC debido a la naturaleza de los datos. Además, como los datos no son de tipo textual, se selecciona K Nearest Neighbor. Por último, se elige implementar un ensamblado de árboles, específicamente un Random Forest, como estrategia de modelado.

In [ ]:
df_filtered.dropna(subset=["Club_Position"], inplace=True)
labels = df_filtered.loc[:, "Club_Position"]
features = df_filtered.drop(columns=texto)
#80% entrenamiento y 20% test
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.2,random_state=42,stratify=labels)


In [ ]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

pip_svc.fit(X_train, y_train)
y_pred = pip_svc.predict(X_train)

# Obtener los nombres reales de las clases
class_names = pip_svc.classes_

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Generar un informe de clasificación
classification_rep = classification_report(y_train, y_pred, target_names=class_names)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix):
    table.add_row([class_names[i], *row])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con svc:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con svc:")
print(classification_rep)

Matriz de confusión en conjunto de entrenamiento con svc:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   506   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |  583   |    77   |       32       |        0        |    0    |
|     central     |    0    |   74   |   861   |      126       |       125       |   105   |
|  central_ataque |    0    |   62   |   291   |      154       |        10       |    5    |
| central_defensa |    0    |   0    |   197   |       5        |       241       |   150   |
|     defensa     |    0    |   1    |    62   |       0        |        31       |   1861  |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

I

In [ ]:
# Predecir en el conjunto de prueba
y_pred = pip_svc.predict(X_test)

# Calcular la matriz de confusión en el conjunto de prueba
conf_matrix_test = confusion_matrix(y_pred, y_test)

# Generar un informe de clasificación en el conjunto de prueba
classification_rep_test = classification_report(y_pred, y_test, target_names=class_names)

# Crear una tabla para la matriz de confusión en el conjunto de prueba
table_test = pt.PrettyTable()
table_test.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix_test):
    table_test.add_row([class_names[i], *row])

# Imprimir resultados en el conjunto de prueba
print("Matriz de confusión en conjunto de prueba con svc:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con svc:")
print(classification_rep_test)


Matriz de confusión en conjunto de prueba con svc:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   126   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |  140   |    10   |       22       |        0        |    0    |
|     central     |    0    |   24   |   211   |       77       |        55       |    10   |
|  central_ataque |    0    |   8    |    38   |       30       |        0        |    0    |
| central_defensa |    0    |   0    |    33   |       1        |        54       |    6    |
|     defensa     |    0    |   1    |    31   |       1        |        39       |   473   |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

Informe 

In [ ]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

pip_knn.fit(X_train, y_train)
y_pred = pip_knn.predict(X_train)

# Obtener los nombres reales de las clases
class_names = pip_knn.classes_

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Generar un informe de clasificación
classification_rep = classification_report(y_train, y_pred, target_names=class_names)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix):
    table.add_row([class_names[i], *row])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con knn:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con knn:")
print(classification_rep)

Matriz de confusión en conjunto de entrenamiento con knn:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   506   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |  590   |    76   |       21       |        2        |    3    |
|     central     |    0    |   96   |   956   |       67       |        75       |    97   |
|  central_ataque |    0    |   78   |   240   |      178       |        14       |    12   |
| central_defensa |    0    |   3    |   178   |       8        |       279       |   125   |
|     defensa     |    0    |   4    |   103   |       0        |        61       |   1787  |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

I

In [ ]:
# Predecir en el conjunto de prueba
y_pred = pip_knn.predict(X_test)

# Calcular la matriz de confusión en el conjunto de prueba
conf_matrix_test = confusion_matrix(y_pred, y_test)

# Generar un informe de clasificación en el conjunto de prueba
classification_rep_test = classification_report(y_pred, y_test, target_names=class_names)

# Crear una tabla para la matriz de confusión en el conjunto de prueba
table_test = pt.PrettyTable()
table_test.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix_test):
    table_test.add_row([class_names[i], *row])

# Imprimir resultados en el conjunto de prueba
print("Matriz de confusión en conjunto de prueba con knn:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con knn:")
print(classification_rep_test)


Matriz de confusión en conjunto de prueba con knn:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   126   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |  128   |    27   |       36       |        3        |    1    |
|     central     |    0    |   35   |   202   |       71       |        54       |    24   |
|  central_ataque |    0    |   9    |    25   |       19       |        2        |    1    |
| central_defensa |    0    |   0    |    34   |       2        |        52       |    25   |
|     defensa     |    0    |   1    |    35   |       3        |        37       |   438   |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

Informe 

In [ ]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

pip_rf.fit(X_train, y_train)
y_pred = pip_rf.predict(X_train)

# Obtener los nombres reales de las clases
class_names = pip_rf.classes_

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Generar un informe de clasificación
classification_rep = classification_report(y_train, y_pred, target_names=class_names)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix):
    table.add_row([class_names[i], *row])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con rf:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con rf:")
print(classification_rep)

Matriz de confusión en conjunto de entrenamiento con rf:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   506   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |  692   |    0    |       0        |        0        |    0    |
|     central     |    0    |   0    |   1291  |       0        |        0        |    0    |
|  central_ataque |    0    |   0    |    0    |      522       |        0        |    0    |
| central_defensa |    0    |   0    |    0    |       0        |       593       |    0    |
|     defensa     |    0    |   0    |    0    |       0        |        0        |   1955  |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

In

In [ ]:
# Predecir en el conjunto de prueba
y_pred = pip_rf.predict(X_test)

# Calcular la matriz de confusión en el conjunto de prueba
conf_matrix_test = confusion_matrix(y_pred, y_test)

# Generar un informe de clasificación en el conjunto de prueba
classification_rep_test = classification_report(y_pred, y_test, target_names=class_names)

# Crear una tabla para la matriz de confusión en el conjunto de prueba
table_test = pt.PrettyTable()
table_test.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix_test):
    table_test.add_row([class_names[i], *row])

# Imprimir resultados en el conjunto de prueba
print("Matriz de confusión en conjunto de prueba con rf:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con rf:")
print(classification_rep_test)


Matriz de confusión en conjunto de prueba con rf:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   126   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |  142   |    12   |       19       |        0        |    0    |
|     central     |    0    |   25   |   228   |       84       |        60       |    10   |
|  central_ataque |    0    |   6    |    18   |       25       |        0        |    0    |
| central_defensa |    0    |   0    |    20   |       1        |        43       |    4    |
|     defensa     |    0    |   0    |    45   |       2        |        45       |   475   |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

Informe d

Al resumir las metricas obtenidas en los conjuntos de prueba:
1.   SVC:
  - Precisión promedio (macro avg): 0.67
  - Recall promedio (macro avg): 0.70
  - F1-score promedio (macro avg): 0.68
  - Accuracy: 0.74

2.   KNN:
  - Precisión promedio (macro avg): 0.63
  - Recall promedio (macro avg): 0.64
  - F1-score promedio (macro avg): 0.62
  - Accuracy: 0.69

3.   Random Forest:
  - Precisión promedio (macro avg): 0.66
  - Recall promedio (macro avg): 0.73
  - F1-score promedio (macro avg): 0.67
  - Accuracy: 0.75


En función de estos resultados, el modelo Random Forest parece ser el mejor entre los tres modelos evaluados, ya que tiene la precisión, el recall y el F1-score más altos en promedio (macro avg). Además, tiene la mayor precisión general (accuracy) y el F1-score ponderado (weighted avg). Por lo tanto, en términos de rendimiento global, el modelo Random Forest es el mejor en este escenario.

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

# Coeficiente de Determinación (R²)

El coeficiente *R²* es una métrica estadística utilizada en el análisis de regresión para evaluar la bondad de ajuste de un modelo de regresión. Esta métrica proporciona información sobre cuánta variabilidad en la variable dependiente puede ser explicada por el modelo de regresión.

La fórmula general para calcular *R²* es:

$$ R² = 1 - \frac{SSR}{SST} $$

Donde:

- *R²* es el coeficiente de determinación.
- *SSR* (Sum of Squares Residual) es la suma de los cuadrados de las diferencias entre los valores observados y los valores predichos por el modelo.
- *SST* (Sum of Squares Total) es la suma de los cuadrados de las diferencias entre los valores observados y la media de los valores observados.

 Un *R²* de 1 indica que el modelo explica toda la variabilidad, mientras que un *R²* de 0 significa que el modelo no explica nada de la variabilidad y se ajusta tan mal como simplemente usar la media de los valores observados para hacer predicciones.



In [ ]:
df_salarios = pd.read_csv('/content/drive/MyDrive/lab8/salarios.csv')
df_salarios

,Unnamed: 0,Player,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0
...,...,...,...
2529,1527,Simone Ghidotti,283.0
2530,2539,Valentin Gendrey,100.0
2531,1001,John Yeboah,70.0
2532,1000,Antonis Aidonis,70.0


Lo primero que se observa es que los dataframes no tienen igual cantidad de registros, asi que se procede a hacer un cruce con los datos coincidentes.

In [ ]:
df_players_salarios = df_salarios.merge(df_players, left_on='Player', right_on='Name')

In [ ]:
# Contar valores nulos por columna
valores_nulos_por_columna = df_players_salarios.isnull().sum()

# Imprimir el recuento de valores nulos por columna
print(valores_nulos_por_columna)

Unnamed: 0              0
Player                  0
Weekly Salary           0
Name                    0
Nationality             0
National_Position    1527
Club_Position           0
Height                  0
Weight                  0
Preffered_Foot          0
Age                     0
Work_Rate               0
Weak_foot               0
Skill_Moves             0
Ball_Control            0
Dribbling               0
Marking                 0
Sliding_Tackle          0
Standing_Tackle         0
Aggression              0
Reactions               0
Interceptions           0
Vision                  0
Composure               0
Crossing                0
Short_Pass              0
Long_Pass               0
Acceleration            0
Speed                   0
Stamina                 0
Strength                0
Balance                 0
Agility                 0
Jumping                 0
Heading                 0
Shot_Power              0
Finishing               0
Long_Shots              0
Curve       

In [ ]:
df_players_salarios.drop(columns=['Player'], inplace=True)


En primer lugar, se analizan los diferentes tipos de variables y se clasifican en cuatro categorías: texto, numéricas, categóricas u ordinales. A continuación, se decide aplicar el Standard Scaler a las variables numéricas, dado que su distribución se asemeja a una distribución normal. Por otro lado, para las columnas categóricas se opta por utilizar el One-Hot Encoder, ya que representan categorías sin un orden específico. Por último, se elige el Ordinal Encoder para la variable Week_foot y Skill_Moves.

Siguiendo la guía, notamos que estamos realizando predicciones con menos de 100,000 observaciones y un gran número de características, por lo que decidimos probar con Ridge Regression.

In [ ]:
from sklearn.linear_model import Ridge

numericas = ['Age','Weight','Height','Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
           'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
           'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
           'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
           'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
           'Freekick_Accuracy', 'Penalties', 'Volleys']

categoricas = ['Preffered_Foot','Work_Rate','Club_Position','Seleccionado']
ordinal = ['Weak_foot','Skill_Moves']
texto = ['Name','National_Position','Nationality']

ct = ColumnTransformer(
    [
        ('One Hot',OneHotEncoder(sparse=False, handle_unknown="ignore",drop='if_binary'),categoricas),
        ('Ordinal',OrdinalEncoder(),ordinal),
        ('Scaler',MinMaxScaler(),numericas)
    ],
    remainder='passthrough'
)

Ridge_reg = Pipeline(steps=[
                      ("preprocessing", ct),
                      ("model", Ridge())
])

In [ ]:
from sklearn.metrics import r2_score

labels = df_players_salarios.loc[:, "Weekly Salary"]
features = df_players_salarios.drop(columns=texto + ['Weekly Salary'])
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Fit
Ridge_reg.fit(X_train, y_train)

# Predict
y_train_pred = Ridge_reg.predict(X_train)
y_test_pred = Ridge_reg.predict(X_test)

# Resultados
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print("R^2 en el conjunto de entrenamiento: {:.2f}".format(r2_train))
print("R^2 en el conjunto de prueba: {:.2f}".format(r2_test))

R^2 en el conjunto de entrenamiento: 0.35
R^2 en el conjunto de prueba: 0.24


Al probar Ridge Regression, notamos que los resultados no cumplen con los requisitos establecidos por Renacin. Siguiendo sus indicaciones, procedemos a considerar la siguiente opción, que corresponde a Ensemble Regressor. En donde se usara RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = Pipeline(steps=[
                      ("preprocessing", ct),
                      ("model", RandomForestRegressor(random_state=42))
])

# Fit
rf_reg.fit(X_train, y_train)

# Predict
y_train_pred = rf_reg.predict(X_train)
y_test_pred = rf_reg.predict(X_test)

# Resultados para el modelo de Random Forest Regressor
r2_train_rf = r2_score(y_train, y_train_pred)
r2_test_rf = r2_score(y_test, y_test_pred)

print("R^2 en train (Random Forest Regressor): {:.2f}".format(r2_train_rf))
print("R^2 en test (Random Forest Regressor): {:.2f}".format(r2_test_rf))

R^2 en train (Random Forest Regressor): 0.96
R^2 en test (Random Forest Regressor): 0.58


No le fallamos a Renacin <3

El modelo de regresión de Random Forest logra un R^2 de 0.58 en el conjunto de prueba, superando el puntaje de 0.24 obtenido por la regresión Ridge. Esta superioridad podría atribuirse a la capacidad de los bosques aleatorios para modelar relaciones no lineales y de alta dimensión entre las características. En contraste, la regresión Ridge es un modelo lineal que asume una relación lineal entre las características y la variable objetivo. Si las relaciones subyacentes en los datos son verdaderamente no lineales, los modelos basados en árboles, como el bosque aleatorio, tienen más probabilidades de capturar estas relaciones de manera efectiva en comparación con los modelos lineales, como Ridge.

Para mejorar aún más el rendimiento del modelo, se podría considerar la adición de más datos, ya que al cruzar los datos de las bases de datos de salarios y jugadroes se obtiene un total de 1,866 filas, loo que es poco. Además, se podría aumentar la cantidad de características utilizadas, como incorporar caracteristicas de rendimiento de los jugadores y variables economicas que afecten al pais en que esta jugando el jugador.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>